In [48]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

## parse DDS data

### Text from DiscoverDataScience.org pasted into Google sheets & saved as CSVs:

source: https://www.discoverdatascience.org/programs/bachelors-in-data-science/

CSV paste: https://docs.google.com/spreadsheets/d/1lIVbtNU8OUB9fkqfGqM1-fYBWlQvkMUFKS9GLsjC-J4/edit?usp=sharing

In [49]:
out = pd.DataFrame() # to build

## phd

In [50]:
def sort_data(deg):
    progs = pd.read_csv(f'dds_{deg}_progs.csv',header=None)[0]
    prog_data = []
    p = []
    for line in progs:
        if 'Offerings' in line:
            prog_data.append(p)
            p = []
        else:
            p.append(line)
    df = pd.DataFrame(prog_data)
    df = df.dropna(subset=[0])
    return df

def parse_phds(df, fancy):
    df['institution'] = df[0].apply(lambda x: x.split('–')[0].strip())
    df['location'] = df[0].apply(lambda x: x.split('–')[1].strip())
    df['course_name'] = df[1]
    df.drop(columns=[0, 2, 4], inplace=True)
    df['online'] = df[3].apply(lambda x: 'Y' if 'online' in x else np.nan)
    df['in_person'] = df[3].apply(lambda x: 'Y' if 'campus' in x.lower() else np.nan)
    df['cost'] = df[5].apply(lambda x: x.split('tion:')[1].split(')')[0]+')')
    df['cost'] = df['cost'].apply(lambda x: x if '(' in x else x.replace(')', ''))
    df.drop(columns=[1,3,5], inplace=True)

    df['class'] = fancy
    return df

In [51]:
df = parse_phds(sort_data('phd'), 'Ph.D.')
out = pd.concat([out, df], sort=False)

### Bachelor's

In [52]:
def sort_data(deg):
    progs = pd.read_csv(f'dds_{deg}_progs.csv',header=None)[0]
    prog_data = []
    p = []
    for line in progs:
        if 'offerings' in line.lower():
            prog_data.append(p)
            p = []
        elif 'minor' not in line.lower() or 'Major or Minor' in line or 'and Data Science Minor' in line:
            p.append(line)
    df = pd.DataFrame(prog_data)
    df = df.dropna(subset=[0])
    return df
df = sort_data('ba')

In [53]:
data = []

for i, row in df.iterrows():
    r_data = {}
    
    for ci, val in enumerate(row):
        
        if 'Program Length:' in str(val):
            r_data['duration'] = val.split(': ')[1]
        elif 'Delivery Method:' in str(val): 
            method = val.split(': ')[1]
            if 'campus' in method.lower():
                r_data['online'] = 'Y'
            if 'online' in method.lower():
                r_data['in_person'] = 'Y'
        elif 'Tuition:' in str(val): 
            r_data['cost'] = val.split(': ')[1].split(')')[0]+')'
            if '(' not in r_data['cost']:
                r_data['cost'] = r_data['cost'].replace(')', '')
            
        elif ci==0: 
            r_data['insti_loc'] = val
        elif ci==1:
            r_data['course_name'] = val
    
    data.append(r_data)
    
df = pd.DataFrame(data)
        
df['institution'] = df['insti_loc'].apply(lambda x: x.split('–')[0].split('-')[0].strip())
df['location'] = df['insti_loc'].apply(lambda x: x.split('–')[-1].split('-')[-1].strip())
df.drop(columns=['insti_loc'], inplace=True)

df['class'] = "Bachelor's"
out = pd.concat([out, df], sort=False)

### Master's

In [54]:
def sort_data(deg):
    progs = pd.read_csv(f'dds_{deg}_progs.csv',header=None)[0]
    prog_data = []
    p = []
    for line in progs:
        if 'offering' in line.lower():
            prog_data.append(p)
            p = []
        elif 'minor' not in line.lower(): # or 'Major or Minor' in line or 'and Data Science Minor' in line:
            p.append(line)
    df = pd.DataFrame(prog_data)
    df = df.dropna(subset=[0])
    return df
df = sort_data('ma')

In [55]:
data = []

for i, row in df.iterrows():
    r_data = {}
    
    for ci, val in enumerate(row):
        
        if 'Program Length:' in str(val):
            r_data['duration'] = val.split(': ')[1]
        elif 'Delivery Method:' in str(val): 
            method = val.split(': ')[1]
            if 'campus' in method.lower():
                r_data['online'] = 'Y'
            if 'online' in method.lower():
                r_data['in_person'] = 'Y'
        elif 'Tuition:' in str(val): 
            r_data['cost'] = val.split(': ')[1].split(')')[0]+')'
            if '(' not in r_data['cost']:
                r_data['cost'] = r_data['cost'].replace(')', '')
            
        elif ci==0: 
            r_data['insti_loc'] = val
        elif ci==1:
            r_data['course_name'] = val
    
    data.append(r_data)
    
df = pd.DataFrame(data)
        
df['institution'] = df['insti_loc'].apply(lambda x: x.split('–')[0].split('-')[0].strip())
df['location'] = df['insti_loc'].apply(lambda x: x.split('–')[-1].split('-')[-1].strip())
df.drop(columns=['insti_loc'], inplace=True)

df['class'] = "Master's"
out = pd.concat([out, df], sort=False)

#### combined DDS data:

In [56]:
out.head()

,institution,location,course_name,online,in_person,cost,class,duration
0,Boise State University,"Boise, Idaho",PhD in Computing – Data Science Concentration,NaN,Y,$470 per credit (Resident),Ph.D.,NaN
1,Bowling Green State University,"Bowling Green, Ohio",Ph.D. in Data Science,NaN,Y,$445 per credit (Ohio Resident),Ph.D.,NaN
2,Brown University,"Providence, Rhode Island",PhD in Computer Science – Concentration in Dat...,NaN,Y,"$66,702 per year",Ph.D.,NaN
3,Chapman University,"Irvine, California",Doctorate in Computational and Data Sciences,NaN,Y,"$1,630 per credit",Ph.D.,NaN
4,Clemson University / Medical University of Sou...,Joint Program,Doctor of Philosophy in Biomedical Data Scienc...,NaN,Y,$668 per credit ( South Carolina Resident),Ph.D.,NaN


---

# join with Portal data

In [57]:
uni_locs = pd.read_csv('portal_data.csv')
uni_locs.institution = uni_locs.institution.str.replace('INTO ', '')

### identify and drop duplicates

In [58]:
out = out.reset_index(drop=True)
to_drop = []
appended = 0

# fuzzy string matching algorithm to judge similarity between rows

# for tuning parameters  
verb = False
match_verb = False

for uni_i, i in tqdm(enumerate(uni_locs.course_name.astype(str))):
    u_inst = uni_locs.loc[uni_i, 'institution']
    u_class = uni_locs.loc[uni_i, 'class']
    u_loc = uni_locs.loc[uni_i, 'location']
    for new_i, j in enumerate(out.course_name.astype(str)):
        n_inst = out.loc[new_i, 'institution']
        n_class = out.loc[new_i, 'class']
        n_loc = out.loc[new_i, 'location']
        fscore1 = fuzz.partial_ratio(i.lower(),j.lower())
        fscore2 = fuzz.ratio(u_inst.lower(),n_inst.lower())
        class_match = u_class == n_class
        
        # tune these with verb=True :
        if (abs(fscore1-77)<5 or abs(fscore2-88)<5) and verb:
                print(u_class, '|', u_inst, '-', i, f'({u_loc})')
                print(n_class, '|', n_inst, '-', j, f'({n_loc})')
                if fscore1 > 75: fscore1_meet = 'meets'
                else: fscore1_meet = 'miss'
                if fscore2 > 88: fscore2_meet = 'meets'
                else: fscore2_meet = 'miss'
                print(f'course: {fscore1} ({fscore1_meet}) | school: {fscore2} ({fscore2_meet})')
                print()
        if fscore1>77 and fscore2>88 and class_match and ('Arizona' not in u_inst):

            # MATCH
            if match_verb:
                print('MATCH!')
                print(fscore1, fscore2)
                print(u_class, '|', u_inst, '-', i, f'({u_loc})')
                print(n_class, '|', n_inst, '-', j, f'({n_loc})')
                print()
            
            out = out.rename(columns={'duration':'dur_number'})
            
            for col in out.columns:
                if str(uni_locs.loc[uni_i, col]).lower()=='nan':
                    if str(out.loc[new_i, col]).lower()!='nan':
                        uni_locs.loc[uni_i, col] = out.loc[new_i, col]
                        appended +=1
            
            to_drop.append(new_i)

out = out.drop(to_drop)         

##  combine + save 

In [63]:
comb = pd.concat([uni_locs, out], sort=False)
comb.to_csv('all_degs.csv', index=False)